In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import re
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


# To set up a temporary directory for caching pipeline results
from tempfile import mkdtemp

# To build a pipeline
from sklearn.pipeline import Pipeline

# To do a cross-validated grid search
from sklearn.model_selection import GridSearchCV

In [8]:
def sel_col_model(df,reg_exp):
    '''
    This function return a filtered dataframe that contains given (reg_exp) in the column name
    Census_Tract and Year are set as index (if present)
    '''
    temp_df=df.copy()
    #df['Census_Tract']=df['Census_Tract'].astype('int32')        
    #temp_df=df.set_index(['Census_Tract','YEAR'],drop=True)
    new_df=temp_df.loc[:,[bool(re.search(reg_exp,col)) for col in temp_df]]
    if 'Census_Tract' in temp_df.columns:
        new_df=pd.concat([temp_df['Census_Tract'],new_df],axis=1)
    if 'YEAR' in temp_df.columns:
        new_df=pd.concat([temp_df['YEAR'],new_df],axis=1)
    
    return new_df

In [5]:
from functions import *

* **Pearson Correlation Test**
* **Scatter Plots to Test for Linear Relationships**


In [15]:
import statsmodels.api as sm
from scipy.stats import norm
from scipy import stats
from scipy.stats import pearsonr

In [7]:
df=pd.read_csv('../data/interim/baseline_model.csv',index_col=0)

In [12]:
sel_col_model(df,'Median')

,YEAR,Census_Tract,Median_Income_2010,Median_Income_2011,Median_Income_2012,Median_Income_2013,Median_Income_2014,Median_Income_2015,Median_Income_2016,Median_Income_2017,Median_Income_2018,Median_Income_2019,Median_Income_2020,Median_Income_2021
0,2006,10100,36905.0,31919.0,31063.0,32191.0,30798.0,32188.0,29861.0,33750.0,37985.0,32474.0,42891.0,60316.0
1,2007,10100,36905.0,31919.0,31063.0,32191.0,30798.0,32188.0,29861.0,33750.0,37985.0,32474.0,42891.0,60316.0
2,2008,10100,36905.0,31919.0,31063.0,32191.0,30798.0,32188.0,29861.0,33750.0,37985.0,32474.0,42891.0,60316.0
3,2009,10100,36905.0,31919.0,31063.0,32191.0,30798.0,32188.0,29861.0,33750.0,37985.0,32474.0,42891.0,60316.0
4,2010,10100,36905.0,31919.0,31063.0,32191.0,30798.0,32188.0,29861.0,33750.0,37985.0,32474.0,42891.0,60316.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12421,2019,844700,31204.5,28496.0,21358.0,27439.5,22994.0,22792.5,24070.0,25552.5,24206.5,33996.5,31905.0,31898.0
12422,2020,844700,31204.5,28496.0,21358.0,27439.5,22994.0,22792.5,24070.0,25552.5,24206.5,33996.5,31905.0,31898.0
12423,2021,844700,31204.5,28496.0,21358.0,27439.5,22994.0,22792.5,24070.0,25552.5,24206.5,33996.5,31905.0,31898.0
12424,2022,844700,31204.5,28496.0,21358.0,27439.5,22994.0,22792.5,24070.0,25552.5,24206.5,33996.5,31905.0,31898.0


In [13]:
sel_col_model(df,'Median')
y_change=df['Median_Income_2021']-df['Median_Income_2010']

df_temp=df.drop(columns=sel_col_model(df,'Median').columns[1:]) #starting from index 1 to exlude year 2010
df_temp=df_temp.drop(columns='CENSUS_TRACT') #drop grouping columns not relevant for the prediction

In [16]:
pearson_test=pd.DataFrame({'feature':[],'corr_coeff':[],'p_value':[]})
k=0
for i in df_temp.columns:
    x=df_temp[i]
    corr_coef, p_value = pearsonr(x, y_change)
    pearson_test.loc[k]=[i,corr_coef,p_value]
    k+=1

In [17]:
pearson_test[pearson_test['p_value']>=0.05]

,feature,corr_coeff,p_value
0,YEAR,-0.001472,0.869699
12,PERMIT_TYPE_FOR EXTENSION OF PMT,-0.010481,0.242719
15,PERMIT_TYPE_REINSTATE REVOKED PMT,-0.007236,0.419902
25,REVIEW_TYPE_ELECTRICAL PLAN REVIEW,-0.000810,0.928067
30,REVIEW_TYPE_TRADITIONAL DEVELOPER SERVICES,0.002944,0.742773
32,CONTACT_1_TYPE_BUILDING OWNER,0.011860,0.186186
39,CONTACT_1_TYPE_CONTRACTOR-VENTILATION,0.000967,0.914160
42,CONTACT_1_TYPE_MASON - BRICK AND CONCRETE,0.006422,0.474129
43,CONTACT_1_TYPE_MASON - BRICK ONLY,0.017200,0.055203
44,CONTACT_1_TYPE_MASON - CONCRETE ONLY,0.005475,0.541725


In [19]:
list(pearson_test[pearson_test['p_value']>=0.05]['feature'])

col_drop=list(pearson_test[pearson_test['p_value']>=0.05]['feature'])

#Drop columns where p value is 0.05 or greater
df_lin_model=df.drop(columns=col_drop)

In [22]:
df_lin_model

,Census_Tract,PROCESSING_TIME,BUILDING_FEE_PAID,ZONING_FEE_PAID,OTHER_FEE_PAID,SUBTOTAL_PAID,TOTAL_FEE,REPORTED_COST,CENSUS_TRACT,Median_Income_2010,...,CONTACT_1_TYPE_EXPEDITOR,CONTACT_1_TYPE_MASONRY CONTRACTOR,CONTACT_1_TYPE_OWNER AS ARCHITECT & CONTRACTR,CONTACT_1_TYPE_OWNER AS GENERAL CONTRACTOR,CONTACT_1_TYPE_OWNER OCCUPIED,CONTACT_1_TYPE_PLUMBING CONTRACTOR,CONTACT_1_TYPE_SELF CERT ARCHITECT,CONTACT_1_TYPE_SIGN CONTRACTOR,CONTACT_1_TYPE_STRUCTURAL ENGINEER,CONTACT_1_TYPE_TENT CONTRACTOR
0,10100,17.348382,413.985556,37.777778,10.555556,493.299778,599.616667,48492.866667,10100.0,36905.0,...,0.000000,0.000000,0.0,0.177778,0.000000,0.0,0.022222,0.022222,0.022222,0.0
1,10100,22.516667,677.990417,46.041667,6.666667,775.550833,817.962500,54027.933333,10100.0,36905.0,...,0.000000,0.000000,0.0,0.133333,0.000000,0.0,0.000000,0.000000,0.016667,0.0
2,10100,19.258065,546.300000,54.838710,4.838710,684.110968,733.522581,40469.000000,10100.0,36905.0,...,0.032258,0.064516,0.0,0.129032,0.064516,0.0,0.000000,0.000000,0.000000,0.0
3,10100,37.666667,245.153333,42.261905,1.190476,288.605714,303.614524,7894.880952,10100.0,36905.0,...,0.047619,0.000000,0.0,0.142857,0.023810,0.0,0.000000,0.023810,0.190476,0.0
4,10100,9.812500,377.141094,44.531250,7.031250,459.476562,474.984062,18547.517812,10100.0,36905.0,...,0.062500,0.125000,0.0,0.062500,0.031250,0.0,0.031250,0.031250,0.031250,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12421,844700,21.800000,437.116667,54.166667,5.000000,542.450000,550.783333,30958.900000,844700.0,31204.5,...,0.066667,0.000000,0.0,0.033333,0.100000,0.0,0.033333,0.000000,0.000000,0.0
12422,844700,28.125000,355.068333,43.229167,0.000000,406.110000,406.110000,30187.375000,844700.0,31204.5,...,0.041667,0.000000,0.0,0.041667,0.208333,0.0,0.000000,0.041667,0.000000,0.0
12423,844700,13.555556,517.115556,33.333333,8.333333,635.617778,666.652593,36729.222222,844700.0,31204.5,...,0.000000,0.037037,0.0,0.148148,0.037037,0.0,0.148148,0.000000,0.037037,0.0
12424,844700,8.588235,517.116471,22.794118,15.441176,633.208824,702.378235,52383.617647,844700.0,31204.5,...,0.029412,0.000000,0.0,0.000000,0.088235,0.0,0.058824,0.000000,0.029412,0.0


In [20]:
sel_col_model(df,'Median').columns[1:]

Index(['Census_Tract', 'Median_Income_2010', 'Median_Income_2011',
       'Median_Income_2012', 'Median_Income_2013', 'Median_Income_2014',
       'Median_Income_2015', 'Median_Income_2016', 'Median_Income_2017',
       'Median_Income_2018', 'Median_Income_2019', 'Median_Income_2020',
       'Median_Income_2021'],
      dtype='object')

In [24]:
pearson_test=pd.DataFrame({'feature':[],'corr_coeff':[],'p_value':[]})
k=0
for i in df_lin_model.columns:
    x=df_lin_model[i]
    corr_coef, p_value = pearsonr(x, y_change)
    pearson_test.loc[k]=[i,corr_coef,p_value]
    k+=1

pearson_test.sort_values('corr_coeff',ascending=False)

,feature,corr_coeff,p_value
20,Median_Income_2021,0.849537,0.000000e+00
19,Median_Income_2020,0.786577,0.000000e+00
18,Median_Income_2019,0.724467,0.000000e+00
17,Median_Income_2018,0.704508,0.000000e+00
16,Median_Income_2017,0.667216,0.000000e+00
15,Median_Income_2016,0.655315,0.000000e+00
14,Median_Income_2015,0.625634,0.000000e+00
13,Median_Income_2014,0.596395,0.000000e+00
12,Median_Income_2013,0.577080,0.000000e+00
11,Median_Income_2012,0.567350,0.000000e+00


In [6]:
def abs_income_change(df,year_x,n):

    '''
    year_x - current year (the year we are making the prediction from)
    n - the number of years ahead we are making the prediction
    '''

    #select year_x (current year) income values from df 
    str_current='Median_Income_'+str(year_x)
    income_val_current=list(df[str_current])

    #year for which we are making the prediction (future year)
    year_future=year_x+n
    str_future='Median_Income_'+str(year_future)
    income_val_future=list(df[str_future])

    abs_change_li=[]
    for (a,b) in zip(income_val_current,income_val_future):
        abs_change_li.append(b-a)


    return abs_change_li

In [8]:
scores_df=pd.DataFrame({'prediction_ahead_window':[],'train_on_window':[],'train_score':[],'test_score/r2':[]})
i=0
year_from=2010

#n_window loop
for n in range(1,8):
    #year_last_from - the last year for which we can try to predict given the chosen n
    year_last_from=2021-n

    #t_window loop: predicting using fata from the past 1 to 5 years
    for t in range(1,6):

        year_from=2010
        y_target=[] #instantiate y_target
        df_X_result=pd.DataFrame() #instantiate an empty df for X features
        
        #df_X - should contain only x_features, but keep median income for year_from and YEAR - drop
        #df_X=df.drop(columns=[....]) 

        while year_from<=year_last_from:

            #filter and average
            df_temp=df_window(df,year_from,t)

            #y_target
            y_target.extend(abs_income_change(df_temp,year_from,n))

            #x_features: drop columns with median, apart from 'Median_Income_' for the current year
            current_income='Median_Income_'+str(year_from) #current year
            y_col=[col for col in list(df_temp.columns) if (bool(re.search('Median',col)) & (col!=current_income))]
            df_X=df_temp.drop(columns=y_col)

            #x_features: concat to the existing
            df_X_result=pd.concat([df_X_result,df_temp])
            
            #switch to the next year
            year_from+=1
        
        y=np.array(np.array(y_target)/np.mean(y_target))

        X=df_X_result.reset_index()

        #break
        
        #Split data into test and train based on census tracts
 
        geo_set=set(X['Census_Tract'])
        test_len=int(len(geo_set)*0.25)

        geo_test=random.sample(list(geo_set),k=test_len)
        test_mask=X['Census_Tract'].isin(geo_test)

        y_test=y[test_mask]
        y_train=y[~test_mask]

        X_test=X[test_mask]
        X_train=X[~test_mask]

        scaler=MinMaxScaler()
        X_train_scaled=scaler.fit_transform(X_train)
        X_test_scaled=scaler.transform(X_test)

        my_pca=PCA(n_components=3)
        X_pca_train=my_pca.fit_transform(X_train_scaled)
        X_pca_test=my_pca.transform(X_test_scaled)

        #L1 Classification for the Baseline
        my_linreg=Ridge(alpha=0.01)
        my_linreg.fit(X_pca_train,y_train)
        train_score=my_linreg.score(X_pca_train,y_train)
        test_score=my_linreg.score(X_pca_test,y_test)

        scores_df.loc[i]=[n,t,train_score,test_score]   
        i+=1


In [9]:
scores_df.sort_values('test_score/r2',ascending=False)

,prediction_ahead_window,train_on_window,train_score,test_score/r2
26,6.0,2.0,0.393446,0.446220
34,7.0,5.0,0.414834,0.436821
30,7.0,1.0,0.440640,0.413741
33,7.0,4.0,0.410477,0.412767
25,6.0,1.0,0.402342,0.383943
20,5.0,1.0,0.364545,0.378508
29,6.0,5.0,0.394673,0.376477
22,5.0,3.0,0.358189,0.375903
27,6.0,3.0,0.409899,0.371450
28,6.0,4.0,0.382956,0.371432
